<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2023/blob/main/Week3/CalculateSumOnGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9mw938wu
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9mw938wu
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=5e591314c0b39cb2fe3580db1b5176d83f87319cea60ce2cbbfce3032b2993a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-mxjwmd4l/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
#include <stdio.h>

__global__ void sum(int* a, int* b, int* c, const int N) {
	int i = threadIdx.x + blockDim.x * blockIdx.x;
	if (i<N)
		c[i] = a[i] + b[i];	
	//else 
	//	printf("i > N -> %3d in block: %d\n", i, blockIdx.x);
}

void sum_host(int* a, int* b, int* c, const int N) {
	for(int i=0; i<N; ++i)
		c[i] = a[i] + b[i];	
}


int main() { 
	int* h_a = 0;
	int* h_b = 0;
	int* h_c = 0;
	
	int* d_a = 0;
	int* d_b = 0;
	int* d_c = 0;

	const int N = 1024;
	int numThreadsPerBlock =  128;
	int numBlocksPerGrid = ceilf(N/numThreadsPerBlock + 1);
	printf("Num threads per block: %3d\n", numThreadsPerBlock);
	printf("Num blocks per grid: %3d\n", numBlocksPerGrid);

	size_t size = N * sizeof(int);

	//allocate host memory
	h_a = (int*)malloc(size);	 
	h_b = (int*)malloc(size);	 
	h_c = (int*)malloc(size);	 

	//initialize a, b and c
	for(int i=0;i<N;++i) {
		h_a[i] = i+1;
		h_b[i] = h_a[i]*2;
		h_c[i] = 0;
	}

	//allocate device memory 
	cudaMalloc((void**)&d_a, size);
	cudaMalloc((void**)&d_b, size);
	cudaMalloc((void**)&d_c, size);
	
	//copy host data to device memory
	cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice); 
	cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

	//calculate on host 
	sum_host(h_a, h_b, h_c, N); 
	printf("Sum (host): %d\n", h_c); 
  
	//output result
	/*
	printf("Host calculation result: \n");
	for(int i=0;i<N;++i) {
		printf("%3d + %3d = %3d\n", h_a[i], h_b[i], h_c[i]);
		//clear host result to ensure that the result of device is actually from the kernel
		h_c[i] = 0;
	}
	*/

	//calculate on device
	sum<<<numBlocksPerGrid, numThreadsPerBlock>>>(d_a, d_b, d_c, N);

	//copy result from device to host
	cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

  printf("Sum (device): %d\n", h_c);

	//output result
	/*
	printf("--------------------------------------\n");
	printf("Device calculation result: \n");
	for(int i=0;i<N;++i) {
		printf("%3d + %3d = %3d\n", h_a[i], h_b[i], h_c[i]);
	}
	printf("--------------------------------------\n");
	*/
 

	//delete data allocated on device
	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	//delete host memory
	free(h_a);
	free(h_b);
	free(h_c);

	cudaDeviceReset();
	return 0;
}

Num threads per block: 128
Num blocks per grid:   9
Sum (host): 1922404352
Sum (device): 1922404352

